Variational AutoEncoder (VAE) with CelebA
=========================================
---
Formation Introduction au Deep Learning  (FIDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  

## Episode 1 - Train a model

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard


## Step 1 - Init python stuff

In [1]:
import numpy as np
import sys
from importlib import reload

import modules.vae
import modules.data_generator

reload(modules.data_generator)
reload(modules.vae)

from modules.vae  import VariationalAutoencoder
from modules.data_generator import DataGenerator

VariationalAutoencoder.about()
DataGenerator.about()


FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.24

FIDLE 2020 - DataGenerator
Version              : 0.4


## Step 2 - Prepare data
### 2.1 - Dataset localisation

In [2]:
dataset_dir  = '/bettik/PROJECTS/pr-fidle/datasets/celeba'

train_dir    = f'{dataset_dir}/clusters-xs.train'
test_dir     = f'{dataset_dir}/clusters-xs.test'

### 2.2 - DataGenerator
Ok, everything's perfect, now let's instantiate our generator for the entire dataset.

In [3]:
data_gen = DataGenerator(train_dir, 32, k_size=1)
print(f'Datasize : {len(data_gen)}')

Datasize : 24


## Step 3 - Get VAE model

In [4]:
tag = 'CelebA.000'

input_shape = (218, 178, 3)
z_dim       = 200
verbose     = 0

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'}
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':1,  'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/CelebA.000
Config saved in     : ./run/CelebA.000/models/vae_config.json


## Step 4 - Compile it

In [5]:
learning_rate = 0.0005
r_loss_factor = 10000

vae.compile(learning_rate, r_loss_factor)

Compiled.
Optimizer is Adam with learning_rate=0.0005


## Step 5 - Train

In [6]:
batch_size        = 1000
epochs            = 100
image_periodicity = 1      # for each epoch
chkpt_periodicity = 2      # for each epoch
initial_epoch     = 0
dataset_size      = 1

In [7]:
vae.train(data_generator    = data_gen,
          x_test            = np.array([0,1]),
          epochs            = epochs,
          image_periodicity = image_periodicity,
          chkpt_periodicity = chkpt_periodicity,
          initial_epoch     = initial_epoch
         )

Epoch 1/100


InvalidArgumentError:  Incompatible shapes: [32,218,178,3] vs. [32,220,180,1]
	 [[node loss/model_2_loss/sub (defined at /home/paroutyj/.conda/envs/deeplearning2/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_2017]

Function call stack:
keras_scratch_graph
